# Weekly Device Detections Exploration

This notebook explores weekly detection patterns across AudioMoth devices and sites. Detection counts are summarised at both the overall and device level, with additional effort-adjusted metrics used to account for variation in device availability.

## This notebook covers:

- Total detections per week (all devices combined)

- Weekly detections per device

- Effort-adjusted weekly intensity (detections per active device-day)


## Setup System Path And Get Data

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd


# Go up one level to .../audiomoth
PROJECT_ROOT = Path(os.getcwd()).resolve().parent

# Add project root to sys.path so `src` is importable
sys.path.insert(0, str(PROJECT_ROOT))

PROCESSED_DATA_PATH = Path(PROJECT_ROOT) / "data_processed" / "analysis_df.parquet"
analysis_df = pd.read_parquet(PROCESSED_DATA_PATH)

# Make pandas show more columns/rows while exploring
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

## Overall

In [ ]:
weekly_detections = (
    analysis_df.groupby("week").size().rename("detections").reset_index()
)

# active device-days per week overall (count unique device+date pairs)
weekly_active_device_days = (
    analysis_df[["week", "device", "date"]]
    .drop_duplicates()
    .groupby(["week"])
    .size()
    .rename("active_device_days")
    .reset_index()
)

weekly_summary = weekly_detections.merge(
    weekly_active_device_days, on=["week"], how="left"
)

weekly_summary["detections_per_device_day"] = (
    weekly_summary["detections"] / weekly_summary["active_device_days"]
).round(2)


weekly_summary

Effort-adjusted weekly detection intensity increases substantially from Weeks 7–8 through Weeks 12–14, indicating a strong seasonal rise in acoustic activity. Although Week 15 shows the highest per-device-day detection rate, this is based on limited recording effort and may reflect concentrated peak activity rather than sustained intensity.

### Save

In [ ]:
import src.data_store as data_store

data_store.save_dataframe_to_csv(
    weekly_summary,
    Path(PROJECT_ROOT) / "outputs",
    "overall_detections_weekly_summary",
)

### Plot

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))

# Line: effort-adjusted intensity
ax1.plot(weekly_summary["week"], weekly_summary["detections_per_device_day"])
ax1.set_xlabel("Week")
ax1.set_ylabel("Detections per device-day")
ax1.set_title("Weekly Detection Intensity with Recording Effort")

# Secondary axis: effort (device-days)
ax2 = ax1.twinx()
ax2.bar(weekly_summary["week"], weekly_summary["active_device_days"], alpha=0.3)
ax2.set_ylabel("Active device-days")

plt.tight_layout()
plt.show()

## Per Device

In [ ]:
weekly_detections_per_device = (
    analysis_df.groupby(["site", "device", "week"])
    .size()
    .rename("detections")
    .reset_index()
    .sort_values(["site", "device", "week"])
    .reset_index(drop=True)
)

# active device-days per week for each device (count unique device+date pairs)
weekly_active_device_days = (
    analysis_df[["device", "week", "date"]]
    .drop_duplicates()
    .groupby(["device", "week"])
    .size()
    .rename("active_device_days")
    .reset_index()
)

weekly_detections_per_device = weekly_detections_per_device.merge(
    weekly_active_device_days, on=["device", "week"], how="left"
)
weekly_detections_per_device["detections_per_device_day"] = (
    weekly_detections_per_device["detections"]
    / weekly_detections_per_device["active_device_days"]
).round(2)


weekly_detections_per_device.head(50)

### Save

In [ ]:
data_store.save_dataframe_to_csv(
    weekly_detections_per_device,
    Path(PROJECT_ROOT) / "outputs",
    "device_detections_weekly_summary",
)

### Improved Visual Format

In [ ]:
# Pivot the data for a more visually pleasing format.
weekly_detections_wide = weekly_detections_per_device.pivot_table(
    index="device", columns="week", values="detections_per_device_day", fill_value=0
)

weekly_detections_wide

Weekly detection patterns vary between sites but show broadly similar temporal trends when adjusted for the number of active devices. Normalising detections by the number of devices recording each week reduces bias introduced by differences in deployment size and periods of device downtime, allowing more meaningful comparison between sites.

Despite this adjustment, variation in detection levels persists, reflecting differences in local habitat, calling intensity, and temporal availability of recording effort. It should be noted that this approach accounts for whether devices were active during a given week, but does not capture partial-week downtime or variation in daily recording effort, which may still influence weekly detection rates.

### Further analysis 
 
week 9 for device CWT10 sees unusually large detections that doesnt follow an obvious pattern.

In [ ]:
df = analysis_df.copy()
week9_df = df[(df["device"] == "CWT10") & (df["week"] == 9)]
week9_df["common_name"].value_counts().head(10)

The elevated detection rate in week 9 at Breney Farm (CWT10) was largely driven by European Robin, which accounted for approximately 40% of detections that week. This suggests a localised behavioural or territorial pulse rather than a broad community-wide increase in activity.